# Discover GEODES products on a Region Of Interest
In this example we’ll see some ways to discover all images offered by GEODES on a ROI.

## Fix a ROI

In [1]:
import geopandas as gpd
import folium
from shapely.geometry import Polygon

pyr_geom = {"type":"Polygon","coordinates":[[[1,42.9],[1.1,42.9],[1.1,42.8],[1,42.8],[1,42.9]]]}
pyr_gpd = gpd.GeoDataFrame(geometry=[Polygon(pyr_geom['coordinates'][0])], crs="EPSG:4326")

f = folium.Figure(width='100%', height=300)
m = folium.Map(location=[43.2, 1.05], zoom_start=8, tiles='OpenStreetMap').add_to(f)

folium.GeoJson(
    pyr_gpd,
    style_function=lambda feature: {
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(m)
m

## Searching products on ROI
Now, let's look for products that intersect the area of interest for January 2020. To learn more about how to search items, see [the docs](../user_guide/search_items.rst).

In [2]:
from pygeodes import Geodes
geodes = Geodes()

query = {
    "start_datetime":{"gte":"2020-01-01T00:00:00Z"},
    "end_datetime":{"lte":"2020-01-31T23:59:59Z"}
}
sortBy = [{"direction":"asc","field":"eo:cloud_cover"}]
items,dataframe = geodes.search_items(intersects=pyr_geom, query=query, sortBy=sortBy, get_all=True)

/work/scratch/data/robertm/dev/env_tmp/pygeodes_cible/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geodes-portal.cnes.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found 100 items matching your query


  0%|          | 0/1 [00:00<?, ?it/s]/work/scratch/data/robertm/dev/env_tmp/pygeodes_cible/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geodes-portal.cnes.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
                                             

100 item(s) found for query : {'start_datetime': {'gte': '2020-01-01T00:00:00Z'}, 'end_datetime': {'lte': '2020-01-31T23:59:59Z'}}



## Display the image footprints
Let's explore our results on a Folium map.

In [3]:
f = folium.Figure(width='100%', height=500)
m = folium.Map(location=[42.85, 1.05], zoom_start=6, tiles='OpenStreetMap').add_to(f)

tooltip=folium.GeoJsonTooltip(
        fields=['collection'],  
        aliases=['Collection'],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )

# Add the image footprints on the map
folium.GeoJson(
    dataframe,
    style_function=lambda feature: {
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0
    },
    highlight_function=lambda feature: {
        'color': 'yellow',
        'weight': 4,
        'fillOpacity': 0
    },
    tooltip = tooltip
).add_to(m)

# Add the ROI in a different color
folium.GeoJson(
    pyr_gpd,
    style_function=lambda feature: {
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(m)

m

## Display collections in the area
This example illustrates the diversity of data available on GEODES.

In [4]:
# Sort and retrieve unique values of collections
sorted_unique_collections = dataframe["collection"].sort_values().unique()
for i, collection in enumerate(sorted_unique_collections, 1):
    print(f"{i}. {collection}")

1. PEPS_S1_L1
2. PEPS_S1_L2
3. PEPS_S2_L1C
4. THEIA_OSO_RASTER_L3B
5. THEIA_OSO_VECTOR_L3B
6. THEIA_REFLECTANCE_LANDSAT8_L2A
7. THEIA_REFLECTANCE_SENTINEL2_L2A
8. THEIA_REFLECTANCE_SENTINEL2_L3A
9. THEIA_REFLECTANCE_VENUS_VM1_L2A
10. THEIA_REFLECTANCE_VENUS_VM1_L3A
11. THEIA_VENUS_VM1_L1C
